In [16]:
from scipy import stats
import time
# import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
import tqdm
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, PowerTransformer
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import sys

   
sys.path.append("/home/td/Documents/sports_predictor")
sys.path.append("/home/td/Documents/sports_predictor/nba")

from process_data import DataManager

base_url = 'https://www.basketball-reference.com/'
day_scores_base_url = 'https://www.basketball-reference.com/boxscores/?month={month}&day={day}&year={year}'
data_path = r'/media/td/Samsung_T5/sports/nba'
# data_path = r'C:\Users\TristanDelforge\Documents\sports_predictor\nba'
db_name = 'nba_db'
box_score_link_table_name = 'boxscore_links'

box_score_details_table_name = 'boxscore_details'
player_detail_table_name = 'player_details'
box_score_details_table_name_sample = 'boxscore_details_sample'
player_detail_table_name_sample = 'player_details_sample'
processed_team_data_table_name = 'processed_team_data'
processed_player_data_table_name = 'processed_player_data'

processed_team_data_table_name = 'processed_team_data'
processed_player_data_table_name = 'processed_player_data'
aggregated_player_data_table_name = 'aggregated_player_data'
combined_feature_file_data_table_name = 'combined_feature_file'
past_n_game_dataset_table_name = 'past_n_game_dataset'
target = 'win'

date_record_pickle_file_name = 'scraped_dates'
box_score_record_pickle_file_name = 'scraped_games'
max_tries = 5
file_lock = threading.Lock()

starting_rating = 1000
rating_k_factor = 100
rating_floor = 100
rating_ceiling = 10000
rating_d = 1000
k_min_sensitivity = 1

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [19]:
team_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name),sep='|', low_memory=False)
player_data = pd.read_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name), sep='|', low_memory=False)



In [20]:
team_data.shape

(1138, 46)

In [21]:
team_data.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,score_diff,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg
0,den,https://www.basketball-reference.com//teams/DE...,Denver Nuggets,mem,https://www.basketball-reference.com//teams/ME...,Memphis Grizzlies,"FedEx Forum, Memphis, Tennessee",1,3.0,2019,1,28,240,37,84,0.440,8,36,0.222,13,14,0.929,9,28,37,17,10,2,15,18,95,NaN,0.527,0.488,0.429,0.167,21.4,75.7,46.8,45.9,10.7,4.3,14.3,100.0,101.5,98.3
1,mem,https://www.basketball-reference.com//teams/ME...,Memphis Grizzlies,den,https://www.basketball-reference.com//teams/DE...,Denver Nuggets,"FedEx Forum, Memphis, Tennessee",0,-3.0,2019,1,28,240,35,78,0.449,9,32,0.281,13,19,0.684,9,33,42,26,5,7,19,22,92,NaN,0.533,0.506,0.410,0.244,24.3,78.6,53.2,74.3,5.3,14.6,18.0,100.0,98.3,101.5
2,atl,https://www.basketball-reference.com//teams/AT...,Atlanta Hawks,lac,https://www.basketball-reference.com//teams/LA...,Los Angeles Clippers,"STAPLES Center, Los Angeles, California",1,5.0,2019,1,28,240,44,87,0.506,10,34,0.294,25,29,0.862,15,29,44,28,7,6,17,25,123,NaN,0.616,0.563,0.391,0.333,37.5,69.0,53.7,63.6,7.1,10.3,14.6,100.0,125.5,120.4
3,lac,https://www.basketball-reference.com//teams/LA...,Los Angeles Clippers,atl,https://www.basketball-reference.com//teams/AT...,Atlanta Hawks,"STAPLES Center, Los Angeles, California",0,-5.0,2019,1,28,240,41,85,0.482,10,27,0.370,26,28,0.929,13,25,38,27,10,4,16,26,118,NaN,0.606,0.541,0.318,0.329,31.0,62.5,46.3,65.9,10.2,7.5,14.1,100.0,120.4,125.5
4,gsw,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,ind,https://www.basketball-reference.com//teams/IN...,Indiana Pacers,"Bankers Life Fieldhouse, Indianapolis, Indiana",1,32.0,2019,1,28,240,53,98,0.541,13,31,0.419,13,16,0.813,13,35,48,39,9,7,12,19,132,NaN,0.628,0.607,0.316,0.163,31.7,77.8,55.8,73.6,8.9,10.9,10.3,100.0,131.2,99.4


In [6]:
player_data.shape

(712166, 48)

In [7]:
player_data.head()

,ast,ast_pct,blk,blk_pct,day,def_rtg,drb,drb_pct,efg_pct,fg,fg3,fg3_pct,fg3a,fg3a_per_fga_pct,fg_pct,fga,ft,ft_pct,fta,fta_per_fga_pct,location,month,mp,off_rtg,opponent_link,opponent_name,opponent_tag,orb,orb_pct,pf,player_link,player_name,plus_minus,pts,reason,stl,stl_pct,team_link,team_name,team_tag,tov,tov_pct,trb,trb_pct,ts_pct,usg_pct,win,year
0,8.0,41.7,0.0,0.0,1,90.0,4.0,16.1,0.731,8.0,3.0,0.6,5.0,0.385,0.615,13.0,1.0,0.500,2.0,0.154,NaN,11,29:19,133.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,3.0,12.4,2.0,https://www.basketball-reference.com//players/...,Kenny Anderson,20.0,20.0,NaN,3.0,5.5,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,4.0,22.4,7.0,14.3,0.720,25.3,1,1996
1,1.0,5.2,1.0,2.6,1,101.0,2.0,8.1,0.600,8.0,2.0,0.4,5.0,0.333,0.533,15.0,4.0,0.667,6.0,0.400,NaN,11,28:38,130.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,0.0,0.0,2.0,https://www.basketball-reference.com//players/...,Clifford Robinson,15.0,22.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,2.0,10.2,2.0,4.1,0.624,27.8,1,1996
2,1.0,5.2,2.0,5.9,1,98.0,2.0,9.0,0.714,5.0,0.0,NaN,0.0,0.000,0.714,7.0,0.0,0.000,2.0,0.286,NaN,11,25:40,122.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,2.0,9.2,2.0,https://www.basketball-reference.com//players/...,Rasheed Wallace,24.0,10.0,NaN,0.0,0.0,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,1.0,11.3,4.0,9.1,0.635,14.0,1,1996
3,4.0,19.5,1.0,3.2,1,88.0,1.0,4.9,0.400,2.0,0.0,NaN,0.0,0.000,0.400,5.0,4.0,0.800,5.0,1.000,NaN,11,24:17,160.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,0.0,0.0,0.0,https://www.basketball-reference.com//players/...,Aaron McKie,22.0,8.0,NaN,3.0,6.6,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,0.0,0.0,1.0,2.5,0.556,12.3,1,1996
4,2.0,10.8,3.0,9.9,1,83.0,7.0,35.6,0.600,3.0,0.0,0.0,1.0,0.200,0.600,5.0,3.0,0.429,7.0,1.400,NaN,11,22:46,69.0,https://www.basketball-reference.com//teams/VA...,Vancouver Grizzlies,van,3.0,15.7,3.0,https://www.basketball-reference.com//players/...,Arvydas Sabonis,18.0,9.0,NaN,1.0,2.3,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,por,7.0,46.4,10.0,25.8,0.557,26.9,1,1996


In [10]:
team_data_recent = team_data[team_data['year'] >= 2014]
player_data_recent = player_data[player_data['year'] >= 2014]

In [11]:
team_data_recent.shape, player_data_recent.shape

((14806, 45), (188227, 48))

In [30]:
team_data_recent.to_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=box_score_details_table_name_sample),sep='|', index=False)
player_data_recent.to_csv('{data_path}/{db_name}.csv'.format(data_path=data_path,db_name=player_detail_table_name_sample), sep='|', index=False)

In [31]:
def get_processed_data():
    team_df = pd.read_csv(f'{data_path}/{processed_team_data_table_name}.csv', sep = '|', low_memory=False)
    player_df = pd.read_csv(f'{data_path}/{processed_player_data_table_name}.csv', sep = '|', low_memory=False)
    return team_df, player_df

In [32]:
team_df, player_df = get_processed_data()

In [33]:
team_df.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,score_diff,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,home,year_str,month_str,day_str,date_str,key,date_dt,days_since_last_match,team_pregame_rating_0,team_postgame_rating_0,team_pregame_rating_1,team_postgame_rating_1,team_pregame_rating_2,team_postgame_rating_2,team_pregame_rating_3,team_postgame_rating_3
0,orl,https://www.basketball-reference.com//teams/OR...,Orlando Magic,tor,https://www.basketball-reference.com//teams/TO...,Toronto Raptors,"Air Canada Centre, Toronto, Canada",1,1.0,2017,1,29,0.0,42,79,0.532,10,23,0.435,20,26,0.769,6,30,36,25,6,6,12,28,114,NaN,0.630,0.595,0.291,0.329,16.2,76.9,47.4,59.5,6.4,11.8,11.7,100.0,120.8,119.7,0,2017,1,29,2017-01-29,"('2017-01-29', 'orl', 'tor')",2017-01-29,NaN,1000.0,1050.0,1000.0,1050.0,1000,1100,1000.0,1050.0
1,por,https://www.basketball-reference.com//teams/PO...,Portland Trail Blazers,gsw,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,"Moda Center, Portland, Oregon",0,-2.0,2017,1,29,0.0,46,93,0.495,10,27,0.370,9,12,0.750,9,33,42,26,11,2,18,24,111,NaN,0.565,0.548,0.290,0.129,22.5,80.5,51.9,56.5,10.6,3.8,15.5,100.0,106.7,108.6,1,2017,1,29,2017-01-29,"('2017-01-29', 'por', 'gsw')",2017-01-29,NaN,1000.0,950.0,1000.0,950.0,1000,900,1000.0,950.0
2,hou,https://www.basketball-reference.com//teams/HO...,Houston Rockets,ind,https://www.basketball-reference.com//teams/IN...,Indiana Pacers,"Bankers Life Fieldhouse, Indianapolis, Indiana",0,-19.0,2017,1,29,0.0,34,83,0.410,11,39,0.282,22,25,0.880,11,30,41,21,7,6,15,24,101,NaN,0.537,0.476,0.470,0.301,23.9,81.1,49.4,61.8,7.2,9.1,13.8,100.0,103.7,123.2,0,2017,1,29,2017-01-29,"('2017-01-29', 'hou', 'ind')",2017-01-29,NaN,1000.0,950.0,1000.0,950.0,1000,900,1000.0,950.0
3,atl,https://www.basketball-reference.com//teams/AT...,Atlanta Hawks,nyk,https://www.basketball-reference.com//teams/NY...,New York Knicks,"Philips Arena, Atlanta, Georgia",1,3.0,2017,1,29,0.0,52,119,0.437,10,31,0.323,28,40,0.700,17,47,64,36,4,2,14,24,142,NaN,0.520,0.479,0.261,0.336,26.2,75.8,50.4,69.2,3.1,2.6,9.3,100.0,109.4,107.1,1,2017,1,29,2017-01-29,"('2017-01-29', 'atl', 'nyk')",2017-01-29,NaN,1000.0,1050.0,1000.0,1050.0,1000,1100,1000.0,1050.0
4,nyk,https://www.basketball-reference.com//teams/NY...,New York Knicks,atl,https://www.basketball-reference.com//teams/AT...,Atlanta Hawks,"Philips Arena, Atlanta, Georgia",0,-3.0,2017,1,29,0.0,51,128,0.398,18,51,0.353,19,23,0.826,15,48,63,32,9,11,12,39,139,NaN,0.503,0.469,0.398,0.180,24.2,73.8,49.6,62.7,6.9,12.5,8.0,100.0,107.1,109.4,0,2017,1,29,2017-01-29,"('2017-01-29', 'nyk', 'atl')",2017-01-29,NaN,1000.0,950.0,1000.0,950.0,1000,900,1000.0,950.0


In [34]:
team_df_sac = team_df[team_df['team_tag'] == 'sac']
team_df_sac = team_df_sac.sort_values('date_str')

In [35]:
team_df_sac.head()

,team_tag,team_link,team_name,opponent_tag,opponent_link,opponent_name,location,win,score_diff,year,month,day,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,off_rtg,def_rtg,home,year_str,month_str,day_str,date_str,key,date_dt,days_since_last_match,team_pregame_rating_0,team_postgame_rating_0,team_pregame_rating_1,team_postgame_rating_1,team_pregame_rating_2,team_postgame_rating_2,team_pregame_rating_3,team_postgame_rating_3
26,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,phi,https://www.basketball-reference.com//teams/PH...,Philadelphia 76ers,"Wells Fargo Center, Philadelphia, Pennsylvania",0,-3.0,2017,1,30,0.0,41,74,0.554,13,29,0.448,24,29,0.828,9,24,33,31,7,3,20,28,119,NaN,0.686,0.642,0.392,0.392,31.0,72.7,53.2,75.6,7.3,5.8,18.7,100.0,124.6,127.8,0,2017,1,30,2017-01-30,"('2017-01-30', 'sac', 'phi')",2017-01-30,NaN,1000.000000,948.717949,1000.000000,951.475587,1000,800,1000.00,950.000
33,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,hou,https://www.basketball-reference.com//teams/HO...,Houston Rockets,"Toyota Center, Houston, Texas",0,-22.0,2017,1,31,0.0,30,86,0.349,8,29,0.276,15,26,0.577,10,35,45,22,9,7,11,17,83,NaN,0.426,0.395,0.337,0.302,16.9,76.1,42.9,73.3,9.3,17.5,10.1,100.0,86.0,108.8,0,2017,1,31,2017-01-31,"('2017-01-31', 'sac', 'hou')",2017-01-31,1.0,948.717949,898.751710,951.475587,901.520259,800,700,950.00,925.000
81,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,pho,https://www.basketball-reference.com//teams/PH...,Phoenix Suns,"Golden 1 Center, Sacramento, California",0,-2.0,2017,2,3,0.0,38,85,0.447,10,27,0.370,17,22,0.773,6,37,43,23,5,4,13,25,103,NaN,0.544,0.506,0.318,0.259,15.4,86.0,52.4,60.5,5.1,8.3,12.1,100.0,106.0,108.1,1,2017,2,3,2017-02-03,"('2017-02-03', 'sac', 'pho')",2017-02-03,3.0,898.751710,848.821996,901.520259,851.616065,700,600,925.00,962.500
109,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,gsw,https://www.basketball-reference.com//teams/GS...,Golden State Warriors,"Golden 1 Center, Sacramento, California",1,3.0,2017,2,4,0.0,43,88,0.489,7,23,0.304,16,19,0.842,8,38,46,25,11,4,16,18,109,NaN,0.566,0.528,0.261,0.216,19.5,84.4,53.5,58.1,10.6,9.3,14.2,100.0,105.2,102.3,1,2017,2,4,2017-02-04,"('2017-02-04', 'sac', 'gsw')",2017-02-04,1.0,848.821996,906.277782,851.616065,893.018462,600,1400,962.50,1031.250
132,sac,https://www.basketball-reference.com//teams/SA...,Sacramento Kings,chi,https://www.basketball-reference.com//teams/CH...,Chicago Bulls,"Golden 1 Center, Sacramento, California",0,-5.0,2017,2,6,0.0,41,79,0.519,11,28,0.393,14,16,0.875,9,31,40,27,4,5,15,25,107,NaN,0.622,0.589,0.354,0.203,25.0,79.5,53.3,65.9,4.4,8.1,14.8,100.0,116.8,122.2,1,2017,2,6,2017-02-06,"('2017-02-06', 'sac', 'chi')",2017-02-06,2.0,906.277782,859.774796,893.018462,838.146391,1400,700,1031.25,965.625


In [36]:
np.fft.fft(team_df_sac['win'])

array([ 7.90000000e+01+2.55351296e-15j,  3.27120525e-02+1.03247405e+01j,
       -3.72836985e+00-1.93046149e+00j,  1.66641328e+00+2.32271166e+00j,
       -7.23181491e-01-2.54949370e+00j, -9.89654874e-01-2.95250990e+00j,
        2.83625626e+00+4.22033895e+00j,  5.41965580e+00-1.41772644e+00j,
        6.04949455e-02-1.02666200e+00j, -7.87047866e+00-2.75937371e+00j,
       -1.57265128e+00-1.16136603e+01j, -8.10756824e+00+2.67201945e+00j,
       -2.28201993e+00-4.69493707e+00j, -4.79330611e+00-5.24475903e+00j,
       -3.98067605e-01-1.18446098e+00j, -4.90963579e+00-2.51670917e+00j,
       -4.82971327e+00+4.18491315e+00j, -6.91470066e+00+3.88267595e+00j,
       -3.38908481e+00-1.29051779e+00j,  1.21739905e+00+4.70950457e-01j,
        1.08635553e+00-7.52910965e-01j,  3.58324588e+00+1.15360571e+01j,
       -2.86889251e+00+6.88414018e+00j, -1.12761021e+00-1.02124024e+01j,
        2.35499005e-01+3.26211563e+00j, -5.10797341e+00-2.55689640e+00j,
       -5.84546010e+00+5.70547202e+00j, -1.04174273

In [37]:
team_df_sac['win'].shape

(199,)